# Setup

In [1]:
from setting import *
from parameters import *

S = Setting(pbits=50)

In [2]:
def shift_fast(f, shift):
    # compute f(x + shift)
    F = f.base_ring()
    R = f.parent()
    x = f.parent().gen()
    n = f.degree() + 1
    
    # precompute factorials
    if not hasattr(R, "_F_cache"):
        print("Precomputing factorials")
        R._F_cache = [F(1)]
        R._iF_cache = [F(1)]
        
    facts = R._F_cache
    ifacts = R._iF_cache
    for i in range(len(facts),n+1):
        facts.append(facts[i-1] * i)
        ifacts.append(~facts[i])  # can be batched to 1 inversion

    # https://arxiv.org/pdf/0804.2337 p.3
    # (refers to A. V. Aho, K. Steiglitz, and J. D. Ullman. Evaluating
    # polynomials at fixed sets of points. SIAM J. Comp., 4(4):533–539, 1975.
    g = f
    
    g = R([gi * facts[i] for i, gi in enumerate(g)])
    g = R(list(g)[::-1])

    P = R([shift**i * ifacts[i] for i in range(n)])
    # this should be done in FFT or similar
    g = g*P % x**n
    
    g = R(list(g)[::-1])
    g = R([gi * ifacts[i] for i, gi in enumerate(g)])
    
    #assert f(x + shift) == g
    return g

In [3]:
def to_Chebyshev_slow(f):
    R = f.parent()
    x = R.gen()
    n = f.degree()

    # precompute Chebyshev polynomials
    if not hasattr(R, "_T_cache"):
        print("Precomputing Chebyshev polynomials")
        R._T_cache = [f.parent()(1), x]
        R._pow_i2_cache = [f.base_ring()(1), f.base_ring()(1)]

    T = R._T_cache
    for _ in range(len(T), n+1):
        T.append(2*x*T[-1] - T[-2])
        R._pow_i2_cache.append(R._pow_i2_cache[-1] / 2)

    # convert to the basis (naive)
    res = [None] * (n + 1)
    t = f
    for i in reversed(range(n + 1)):
        assert R._pow_i2_cache[i] == 1/T[i].lc(), (i, T[i].lc(), R._pow_i2_cache[i])
        res[i] = t[i] * R._pow_i2_cache[i]
        t -= res[i] * T[i]
    #assert f == sum(res[i] * T[i] for i in range(n + 1))
    return f.parent()(res)

In [4]:
def forward_polys(S, x0, nsteps):
    x = S.Fp2['x'].gen()
    R = x.parent()
    fs = [x - x0.value]
    for i in range(1, nsteps+1):
        f = fs[-1]
        d = f.degree()
        assert f.degree() == 2**(i-1), (f.degree(), i)
        
        gx2 = f(x)*f(-x)  # M(d) mult., 
        g = R(list(gx2)[::2])  # squeeze coefficients
        # g(x**2) = f(x)f(-x)

        # can use Graeffe trick for slight optimization
        #fe = R(list(f)[::2])
        #fo = R(list(f)[1::2])
        #assert f(x) == fe(x**2) + x*fo(x**2)
        #assert g == fe**2 - x*fo**2
    
        gt = to_Chebyshev_slow(g)/2  # M(d) conv.
        #assert g((x+1/x)/2) == gt(x) + gt(1/x)

        #h = R(list(gt)[::-1] + list(gt)[1::1]) + gt[0]*x**d
        h = list(gt)
        h = h[::-1] + h[1::]
        h[d] += gt[0]
        h = R(h)
        
        #assert h == R( gt(x)*x**d + x**d*gt(1/x) )
        assert h.degree() == 2*d
        fs.append(h)
    return fs

def subst_dual_fast(f):
    #f0 = f
    R = f.parent()
    F = R.base_ring()
    f = shift_fast(f, +1) # f(x+1)
    f = R([F(2)**i * c for i, c in enumerate(f)]) # f(2x)
    f = R(list(f)[::-1]) # x^d f(1/x)
    f = shift_fast(f, -1)  # h(f-1)
    return f

def backward_polys(S, x0, nsteps):
    fs = forward_polys(S, x0.dual(), nsteps)
    gs = []
    for i, f in enumerate(fs):
        x = f.parent().gen()
        g = subst_dual_fast(f)
        #assert g == f.subs((x+1)/(x-1)).numerator()
        gs.append(g)
    return gs

In [5]:
S.p.nbits()

51

In [6]:
n = 20
xpath = S.r0.sample_fw(n+1, skip=100)
x0 = xpath[0]
xn = xpath[n]

In [7]:
assert n % 2 == 0

fs = forward_polys(S, x0, n//2)
for i, f in enumerate(fs):
    assert f(xpath[i].value) == 0

gs = backward_polys(S, xn, n//2)
for i, g in enumerate(gs):
    assert g(xpath[-1-i].value) == 0, i

Precomputing Chebyshev polynomials
Precomputing factorials


In [8]:
gcd(fs[-1], gs[-1]).roots(), xpath[n//2]

([(1122759492010968*i + 420563986021176, 1)],
 <Level16:r=1122759492010968*i + 420563986021176>)

In [9]:
print("real answer")
xpath[n//2]

real answer


<Level16:r=1122759492010968*i + 420563986021176>